In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import glob
import mysql.connector
from mysql.connector import Error

In [117]:
query_analytics = 'SELECT * FROM analytics_perf ap WHERE Account = "SB" AND landingPagePath like "%/de/%"'
query_lightspeed = 'SELECT * FROM api_lightspeed_saintbasics_orders WHERE Shipping_Country = "DE"'
query_adwords = 'SELECT aaas.Observation, aac.Name, SUM(aaas.Impr) AS "impressions", SUM(aaas.Clicks) AS "clicks", SUM(aaas.Cost) AS "cost" FROM api_adwords.api_adwords_ag_stats aaas JOIN api_adwords.api_adwords_adgroups aaa on aaa.ID = aaas.Adgroup_ID JOIN api_adwords.api_adwords_campaigns aac on aac.ID = aaa.Campaign_ID JOIN api_adwords.api_adwords_accounts aaa2 on aaa2.ID = aaa.Account_ID WHERE aaa2.ID = "6109709000" GROUP BY 1,2;'
query_facebook = 'SELECT * FROM api_facebook.account_report WHERE Account_ID = "395756991466104"'

In [25]:
try:
    connection = mysql.connector.connect(host='attribution-system-fsg-new.cob86lv75rzo.eu-west-1.rds.amazonaws.com',
                                         database='api_analytics',
                                         user='fsg',
                                         password='Attribution3.0')

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Your connected to database: ", record)
        df_ga = pd.read_sql(query_analytics,con=connection)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.16
Your connected to database:  ('api_analytics',)
MySQL connection is closed


In [112]:
try:
    connection = mysql.connector.connect(host='attribution-system-fsg-new.cob86lv75rzo.eu-west-1.rds.amazonaws.com',
                                         database='api_lightspeed',
                                         user='fsg',
                                         password='Attribution3.0')

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Your connected to database: ", record)
        df_ls = pd.read_sql(query_lightspeed,con=connection)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.16
Your connected to database:  ('api_lightspeed',)
MySQL connection is closed


In [115]:
try:
    connection = mysql.connector.connect(host='attribution-system-fsg-new.cob86lv75rzo.eu-west-1.rds.amazonaws.com',
                                         database='api_adwords',
                                         user='fsg',
                                         password='Attribution3.0')

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Your connected to database: ", record)
        df_aw = pd.read_sql(query_adwords,con=connection)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.16
Your connected to database:  ('api_adwords',)
MySQL connection is closed


In [129]:
try:
    connection = mysql.connector.connect(host='attribution-system-fsg-new.cob86lv75rzo.eu-west-1.rds.amazonaws.com',
                                         database='api_adwords',
                                         user='fsg',
                                         password='Attribution3.0')

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Your connected to database: ", record)
        df_fb = pd.read_sql(query_facebook,con=connection)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.16
Your connected to database:  ('api_adwords',)
MySQL connection is closed


In [ ]:
# Transformations

In [131]:
df_ga['Observation'] = pd.to_datetime(df_ga['Observation'])
df_ga['year_week'] = df.Observation.dt.strftime("%Y-%V")
df_ga['year_month'] = df.Observation.dt.strftime("%Y-%m")

In [132]:
df_ls['Conv_Date'] = pd.to_datetime(df_ls['Conv_Date'])
df_ls['year_week'] = df.Observation.dt.strftime("%Y-%V")
df_ls['year_month'] = df.Observation.dt.strftime("%Y-%m")
df_ls['Conv_Type'] = df_ls['Conv_Type'].fillna('not_returned')
df_ls['Conv_Type'] = df_ls['Conv_Type'].str.replace('exchange - wrong size delivered','returned').str.replace('exchange','returned').str.replace('replace','returned').str.replace('ontbrak van bestelling 2007012','returned')

In [133]:
df_aw['Observation'] = pd.to_datetime(df_aw['Observation'])
df_aw['year_week'] = df.Observation.dt.strftime("%Y-%V")
df_aw['year_month'] = df.Observation.dt.strftime("%Y-%m")

In [134]:
df_fb['Observation'] = pd.to_datetime(df_fb['Observation'])
df_fb['year_week'] = df.Observation.dt.strftime("%Y-%V")
df_fb['year_month'] = df.Observation.dt.strftime("%Y-%m")

In [135]:
# Aggregations

In [136]:
ga_summary_week_all = df_ga.groupby(['year_week']).agg({'transactionRevenue':pd.Series.sum, 'sessions':pd.Series.sum, 'transactions':pd.Series.sum}).reset_index()
ga_summary_week_channels = df_ga.groupby(['year_week','channelGrouping']).agg({'transactionRevenue':pd.Series.sum, 'sessions':pd.Series.sum, 'transactions':pd.Series.sum}).reset_index()
ga_summary_week_all['cvr'] = ga_summary_week_all['transactions']/ga_summary_week_all['sessions']
ga_summary_week_channels['cvr'] = ga_summary_week_channels['transactions']/ga_summary_week_channels['sessions']

In [137]:
ga_summary_month_all = df_ga.groupby(['year_month']).agg({'transactionRevenue':pd.Series.sum, 'sessions':pd.Series.sum, 'transactions':pd.Series.sum}).reset_index()
ga_summary_month_channels = df_ga.groupby(['year_month','channelGrouping']).agg({'transactionRevenue':pd.Series.sum, 'sessions':pd.Series.sum, 'transactions':pd.Series.sum}).reset_index()
ga_summary_month_all['cvr'] = ga_summary_month_all['transactions']/ga_summary_month_all['sessions']
ga_summary_month_channels['cvr'] = ga_summary_month_channels['transactions']/ga_summary_month_channels['sessions']

In [138]:
ls_summary_week_to = df_ls.groupby(['year_week','Conv_Type']).agg({'Revenue_excl_VAT':pd.Series.sum}).unstack(1).fillna(0)
ls_summary_week_tx = df_ls.groupby(['year_week','Conv_Type']).agg({'Conv_ID':pd.Series.nunique}).unstack(1).fillna(0)
ls_summary_week_to.columns = ls_summary_week_to.columns.droplevel()
ls_summary_week_tx.columns = ls_summary_week_tx.columns.droplevel()
ls_summary_week_to.columns = ['TO_EXVAT_EXRET','TO_EXVAT_RET']
ls_summary_week_tx.columns = ['TX_EXRET','TX_RET']
ls_summary_week = ls_summary_week_to.merge(ls_summary_week_tx, on='year_week')

ls_summary_month_to = df_ls.groupby(['year_month','Conv_Type']).agg({'Revenue_excl_VAT':pd.Series.sum}).unstack(1).fillna(0)
ls_summary_month_tx = df_ls.groupby(['year_month','Conv_Type']).agg({'Conv_ID':pd.Series.nunique}).unstack(1).fillna(0)
ls_summary_month_to.columns = ls_summary_month_to.columns.droplevel()
ls_summary_month_tx.columns = ls_summary_month_tx.columns.droplevel()
ls_summary_month_to.columns = ['TO_EXVAT_EXRET','TO_EXVAT_RET']
ls_summary_month_tx.columns = ['TX_EXRET','TX_RET']
ls_summary_month = ls_summary_month_to.merge(ls_summary_month_tx, on='year_month')

In [ ]:
aw_summary_week = df_aw.groupby(['year_week']).agg({'transactionRevenue':pd.Series.sum, 'sessions':pd.Series.sum, 'transactions':pd.Series.sum}).reset_index()

In [141]:
df_aw.head()

,Observation,Name,impressions,clicks,cost,year_week,year_month
0,2020-11-01,NB_Underwear,0.0,0.0,0.0,2020-44,2020-11
1,2020-11-02,NB_Underwear,0.0,0.0,0.0,2020-44,2020-11
2,2020-11-03,NB_Underwear,0.0,0.0,0.0,2020-44,2020-11
3,2020-11-04,NB_Underwear,0.0,0.0,0.0,2020-44,2020-11
4,2020-11-05,NB_Underwear,0.0,0.0,0.0,2020-44,2020-11


In [98]:
ga_summary_month_all

,year_month,transactionRevenue,sessions,transactions,cvr
0,2020-11,1594.62,648,19,0.029321
1,2020-12,2760.70,7312,24,0.003282
2,2021-01,0.00,865,0,0.000000


In [144]:
ls_summary_week

,TO_EXVAT_EXRET,TO_EXVAT_RET,TX_EXRET,TX_RET
year_week,,,,
2020-44,572.2,96.0,11,4
2020-45,1854.0,229.1,41,7


In [85]:
from gspread_pandas import Spread, Client

In [86]:
agu_mt_sheet = Spread('AGU MT Weekly')

In [88]:
# Push Transactions
agu_mt_sheet.df_to_sheet(weekly_summary, index=True, sheet='DataDump_BE', start='A1', replace=True)